# Web Scraping - Lab

## Introduction

Now that you've seen a more extensive example of developing a web scraping script, it's time to further practice and formalize that knowledge by writing functions to parse specific pieces of information from the web page and then synthesizing these into a larger loop that will iterate over successive web pages in order to build a complete dataset.

## Objectives

You will be able to:

* Navigate HTML documents using Beautiful Soup's children and sibling relations
* Select specific elements from HTML using Beautiful Soup
* Use regular expressions to extract items with a certain pattern within Beautiful Soup
* Determine the pagination scheme of a website and scrape multiple pages

## Lab Overview

This lab will build upon the previous lesson. In the end, you'll look to write a script that will iterate over all of the pages for the demo site and extract the title, price, star rating and availability of each book listed. Building up to that, you'll formalize the concepts from the lesson by writing functions that will extract a list of each of these features for each web page. You'll then combine these functions into the full script which will look something like this:  

```python
df = pd.DataFrame()
for i in range(2,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    new_star_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_avails(soup)
    ...
 ```

## Retrieving Titles

To start, write a function that extracts the titles of the books on a given page. The input for the function should be the `soup` for the HTML of the page.

## Retrieve Ratings

Next, write a similar function to retrieve the star ratings on a given page. Again, the function should take in the `soup` from the given HTML page and return a list of the star ratings for the books. These star ratings should be formatted as integers.

## Retrieve Prices

Now write a function to retrieve the prices on a given page. The function should take in the `soup` from the given page and return a list of prices formatted as floats.

## Retrieve Availability

Write a function to retrieve whether each book is available or not. The function should take in the `soup` from a given html page and return a list of the availability for each book.

## Create a Script to Retrieve All the Books From All 50 Pages

Finally, write a script to retrieve all of the information from all 50 pages of the books.toscrape.com website. 

## Level-Up: Write a new version of the script you just wrote. 

If you used URL hacking to generate each successive page URL, instead write a function that retrieves the link from the `"next"` button at the bottom of the page. Conversely, if you already used this approach above, use URL-hacking (arguably the easier of the two methods in this case).

In [1]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup


def get_book_container(soup):
    warning = soup.find('div', class_='alert alert-warning')
    return warning.nextSibling.nextSibling


def get_titles(book_container):
    return [row.find('a')['title'] for row 
            in book_container.findAll('h3')]


def get_ratings(book_container):
    regex = re.compile("^star-rating")
    ratings = [each['class'][-1] for each 
               in book_container.findAll('p', {'class': regex})]
    
    rating_dct = {
        'One': 1, 'Two': 2, 'Three':3, 'Four': 4, 'Five':5
    }
    
    return [rating_dct[rating] for rating in ratings]


def get_prices(book_container):
    return [float(each.text[1:]) for each 
            in book_container.findAll('p', class_='price_color')]


def get_avail(book_container):
    return [each.text.strip() for each 
            in book_container.findAll('p', class_='instock availability')]


def concat_df_lst(df_lst):
    big_df = pd.concat(df_lst)
    for col in big_df.columns:
        big_df[col] = pd.to_numeric(big_df[col], errors='ignore')
    return big_df
    

def create_df():
    print('Working...')
    counter = 0
    new_tag = ''
    df_lst = []
    
    while True:
        url = 'http://books.toscrape.com/'
        url += new_tag
        
        counter += 1

        req_get = requests.get(url)
        soup_curr = BeautifulSoup(req_get.content, 'html')
        
        book_container = get_book_container(soup_curr)
        
        titles = get_titles(book_container)
        ratings = get_ratings(book_container)
        prices = get_prices(book_container)
        avail = get_avail(book_container)
        
        df= pd.DataFrame([titles, ratings, prices, avail]).transpose()
        df.columns = ['title', 'rating', 'price_pounds', 'avail']
        df_lst.append(df)
        
        try:
            new_tag = soup_curr.find('li', class_='next').find('a')['href']
        except AttributeError:
            print(f'Completed with {counter} pages.')
            df = concat_df_lst(df_lst)
            print('DataFrame Shape:', df.shape)
            return df
        
        if 'catalogue' not in new_tag:
            new_tag = 'catalogue/' + new_tag

In [2]:
df = create_df()

Working...
Completed with 50 pages.
DataFrame Shape: (1000, 4)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 19
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         1000 non-null   object 
 1   rating        1000 non-null   int64  
 2   price_pounds  1000 non-null   float64
 3   avail         1000 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 39.1+ KB


## Summary

Well done! You just completed your first full web scraping project! You're ready to start harnessing the power of the web!